In [3]:
import pandas as pd 
import numpy as np
from numpy import array
import os
from pathlib import Path
from collections import namedtuple

### Traversing the folder 

- check the json data
- check the whether the time start and time end are distinct
- check whether there are 5 paragraphs

In [ ]:
t_str = "User015_test11_BDT_11_05-09-2019_recording0_U1567697316_EEG_rawEEGData.csv"

t_str = str.split("_")

In [31]:
File = namedtuple('File', 'name path')
files = []

# p = Path("/cs/home/ybk1/Dissertation/Experiment Anonymised Version")
p = Path("C:\Users\User\OneDrive - University of St Andrews\Modules\CS5099\4. Data Documents\dataset\INSTRUMENTED DIGITAL AND PAPER READINGDATASET\Experiment Anonymised Version")

In [32]:
for item in p.glob('**/*'):
    if item.match('*rawEEGData.csv') and "baseline" not in item.name:
        name = item.name
        path = Path.resolve(item).parent
        
        files.append(File(name,path ))

In [35]:
df = pd.DataFrame(files)

df.to_csv("All EEG files.csv", index=False)

In [26]:
df = pd.DataFrame(files)

### Loading the data 

In [13]:
s_dir = "/cs/home/ybk1/Dissertation/Experiment Anonymised Version/User001_Group_Test_30-08-2019--10-07-00/User001_test1_BET_01_30-08-2019/User001_test1_BET_01_30-08-2019_recording0/"
s_csv = "User001_test1_BET_01_30-08-2019_recording0_U1567156556_EEG_rawEEGData.csv"
s_json = "annotations.json"

In [14]:
s_data = pd.read_csv(s_dir + s_csv)

In [32]:
pd_labels = pd.read_json(s_dir + s_json)

In [31]:
def check_json(labels):
    #check whether the
    if len(labels) != 5:
        print ("There are less than 5 Paragragraphs in the dataset")
    else:
        print("There are 5 paragraphs in the dataset")
    
    cols = ["timeRangeStart", "timeRangeEnd"]
    for col in cols:
        array_length= len(array(labels[col]))
        set_length = len(set(array(labels[col])))
        if array_length == set_length:
            print(col + " has distinct values")
        else: 
            print(col + " does not have distinct values")    
    

In [33]:
check_json(pd_labels)

There are 5 paragraphs in the dataset
timeRangeStart has distinct values
timeRangeEnd has distinct values


In [27]:
labels = array(pd.read_json(s_dir + s_json))
labels

print(len(labels))

5


In [17]:
s_data

,C1,C2,C3,C4,C5,C6,C7,C8,Timestamp,AdjustedUnix
0,172495.625000,-16886.330078,200873.203125,188091.968750,179837.921875,157221.484375,172901.453125,205096.406250,1.106530e+06,1567155478
1,172504.875000,-16903.593750,200863.953125,188080.625000,179839.968750,157228.671875,172908.046875,205104.468750,1.106546e+06,1567155478
2,172504.875000,-16900.970703,200869.828125,188084.484375,179827.421875,157230.437500,172899.984375,205099.328125,1.106552e+06,1567155478
3,172494.234375,-16878.892578,200887.421875,188101.031250,179807.015625,157225.671875,172880.484375,205080.203125,1.106564e+06,1567155478
4,172478.031250,-16841.746094,200913.218750,188126.875000,179780.843750,157215.750000,172855.921875,205053.781250,1.106573e+06,1567155478
...,...,...,...,...,...,...,...,...,...,...
7950,173914.171875,-15843.916992,202884.406250,189654.281250,181753.281250,158724.234375,174455.046875,206533.453125,1.180984e+06,1567155492
7951,173923.171875,-15895.368164,202846.265625,189616.031250,181770.343750,158725.187500,174474.968750,206554.109375,1.180989e+06,1567155492
7952,173942.156250,-15927.840820,202824.515625,189593.000000,181788.562500,158735.390625,174497.203125,206578.468750,1.180995e+06,1567155492
7953,173954.500000,-15934.993164,202822.515625,189589.468750,181798.390625,158746.312500,174507.109375,206589.578125,1.181003e+06,1567155493


In [18]:
# first create zero columns
# then iterate through the each row and check if it is there


In [19]:
def label(timestamp,annotation):
    for row in labels:
        time_start = row[-2]
        time_end = row[-1]
        ann_dict = {"effort": row[-3], "attention": row[-4], "interest": row[-5],"para": row[-6]}
        if timestamp >= time_start and timestamp < time_end:
            return ann_dict[annotation]     
        

In [20]:
def process_data(data):
    annotations = ["effort", "attention", "interest", "para"]
    for ann in annotations:
        data[ann] = data["Timestamp"].apply(label, annotation=ann)
    print("Processed: {}".format(len(data)))
    print("Empty rows \n{}".format(data.isnull().sum()))
    data = data.dropna()
    data.to_csv("pc.csv")
    print(len(data))
    return data   

In [25]:
def generate_datasets(processed):
    X = processed.iloc[:,:9]
    labels = ['interest', 'effort', 'attention']
    for label in labels:
        dataset = pd.concat([X, processed[label]], axis=1).to_csv('EEG_' + label + '_dataset.csv')   


In [26]:
processed = process_data(s_data)
generate_datasets(processed)

Processed: 7955
Empty rows 
C1                 0
C2                 0
C3                 0
C4                 0
C5                 0
C6                 0
C7                 0
C8                 0
Timestamp          0
 AdjustedUnix      0
effort           411
attention        411
interest         411
para             411
dtype: int64
7544
